<a href="https://colab.research.google.com/github/Przeroo/MISSP/blob/main/Challenge1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# pip install pulp
import pulp

# dane
R = {1:8.0, 2:6.0, 3:4.0, 4:10.0, 5:8.0}       # omy
Imax = {1:2.0, 2:3.0, 3:4.0, 4:2.0, 5:2.0}    # A

# problem
prob = pulp.LpProblem("MaxCurrent", pulp.LpMaximize)

# zmienne prądów (non-neg) i napięć (wolne, mogą byc ujemne)
I = {i: pulp.LpVariable(f"I{i}", lowBound=0, upBound=Imax[i]) for i in R}
V_A = pulp.LpVariable("V_A", lowBound=None, upBound=None)   # lewy biegun
V_B = pulp.LpVariable("V_B", lowBound=None, upBound=None)   # węzeł między równoległymi L i R
V_C = pulp.LpVariable("V_C", lowBound=None, upBound=None)   # węzeł po R3
V_D = 0.0                                                  # referencja (prawy biegun)

# prawo Ohma w postaci liniowej: R_i * I_i = V_from - V_to
prob += R[1]*I[1] == V_A - V_B
prob += R[2]*I[2] == V_A - V_B
prob += R[3]*I[3] == V_B - V_C
prob += R[4]*I[4] == V_C - V_D
prob += R[5]*I[5] == V_C - V_D

# bilans prądów w węzłach (konserwacja ładunku)
prob += I[1] + I[2] == I[3]      # węzeł B
prob += I[3] == I[4] + I[5]      # węzeł C

# cel: maksymalizuj prąd dostarczany przez zasilacz (I1+I2)
prob += I[1] + I[2]

# solve
prob.solve(pulp.PULP_CBC_CMD(msg=False))

# wynik
print("Status:", pulp.LpStatus[prob.status])
print("I1..I5 [A]:", [pulp.value(I[i]) for i in sorted(I)])
print("V_A, V_B, V_C [V]:", pulp.value(V_A), pulp.value(V_B), pulp.value(V_C))
print("Suma prądu (wejście) [A]:", pulp.value(I[1] + I[2]))

Status: Optimal
I1..I5 [A]: [1.5428571, 2.0571429, 3.6, 1.6, 2.0]
V_A, V_B, V_C [V]: 42.742857 30.4 16.0
Suma prądu (wejście) [A]: 3.6
